In [46]:
import numpy as np
import tensorflow as tf
import os
import h5py
from PIL import Image

In [47]:
CANCER_TYPES = {
    1: 'Oponiak',
    2: 'Glejak',
    3: 'Guz przysadki'
}

IMAGE_WIDTH = 256
IMAGE_HEIGHT = 256

In [87]:
data_dir = 'data'
mri_images = []
labels = []
borders = []

for file_name in os.listdir(data_dir):
    file_path = os.path.join(data_dir, file_name)
    file_mat = h5py.File(file_path, "r")
    image = np.array(file_mat['/cjdata/image'])
    image = Image.fromarray(image).resize((IMAGE_WIDTH, IMAGE_HEIGHT))
    mri_images.append(np.array(image))
    labels.append(file_mat['/cjdata/label'][0][0])
    borders.append(file_mat['/cjdata/tumorBorder'][0])

In [88]:
mri_images = np.array(mri_images)
labels = np.array(labels, dtype=np.int32) - 1

In [89]:
unique_values, counts = np.unique(labels, return_counts=True)
print('Unique values:', unique_values)
print(len(CANCER_TYPES))

Unique values: [0 1 2]
3


In [90]:
labels

array([2, 2, 2, ..., 2, 1, 2], dtype=int32)

In [91]:
one_hot_labels = np.eye(len(CANCER_TYPES))[labels]

In [92]:
print(one_hot_labels)

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [93]:
mri_images.shape

(3064, 256, 256)

In [94]:
mri_images = np.expand_dims(mri_images, axis=-1)

In [95]:
mri_images.shape

(3064, 256, 256, 1)

In [96]:
from sklearn.model_selection import train_test_split


In [97]:
X_train, X_test, y_train, y_test = train_test_split(
    mri_images,
    one_hot_labels, 
    test_size=0.2, 
    random_state=42
)

In [98]:
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 1)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

/home/michal/mri/.venv/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
/home/michal/mri/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [99]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [100]:
epochs = 50
history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 9s 72ms/step - accuracy: 0.5439 - loss: 1021.8547 - val_accuracy: 0.7341 - val_loss: 0.6856
Epoch 2/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.7438 - loss: 0.6499 - val_accuracy: 0.7406 - val_loss: 0.6441
Epoch 3/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.7760 - loss: 0.5778 - val_accuracy: 0.7439 - val_loss: 0.5682
Epoch 4/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 5s 69ms/step - accuracy: 0.7643 - loss: 0.5862 - val_accuracy: 0.7406 - val_loss: 0.5575
Epoch 5/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.7693 - loss: 0.5326 - val_accuracy: 0.7504 - val_loss: 0.5333
Epoch 6/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.7868 - loss: 0.5180 - val_accuracy: 0.7749 - val_loss: 0.5053
Epoch 7/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.7843 - loss: 0.5394 - val_accuracy: 0.7569 - val_loss: 0.5075
Epoch 8/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.8034 - loss: 0.4894 - val_accuracy: 0.7781 

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

: 